<a href="https://colab.research.google.com/github/RoboTuan/ML_DL_Project/blob/master/Self_supervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import datetime
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from torch.autograd import Function
import torchvision
from torchvision.models import alexnet
from torchvision.utils import make_grid
from sklearn.model_selection import RandomizedSearchCV
import pandas.util.testing as tm
from PIL import Image
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/Copia di GTEA61.zip"
!rm -rf "./__MACOSX"

In [ ]:
#!rm -rf "./ML_DL_Project/"
#!rm -rf "./out_dir/"

In [4]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 204 (delta 127), reused 78 (delta 38), pack-reused 0
Receiving objects: 100% (204/204), 4.32 MiB | 4.01 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [5]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)

./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [6]:
# Se scritti uguali ai parametri della funzione può venire None come risultato
# per questo SEQLEN è maiuscolo 
SEQLEN = 7
DEVICE = 'cuda' # 'cuda' or 'cpu'
NUM_CLASSES = 61
#STAGE = None
train_data_dir = "./GTEA61"
val_data_dir = "./GTEA61_val"
#stage1_dict = None
out_dir = ".out_dir"
trainBatchSize = 32
valBatchSize = 64
numEpochs1 = 3
numEpochs2 = 3
lr1 = 1e-3
lr2 = 1e-4
decay_factor = 0.1
decay_step1 = [25, 75, 150]
decay_step2 = [25, 75]
#Boh?!
MEMSIZE = 512

In [ ]:
#!rm -rf "ML_DL_Project"
#!rm -rf "./GTEA61"
#!rm -rf "./GTEA61_val"

In [7]:
from __future__ import print_function, division
# This is without attention, we must address this better
#from ML_DL_Project.Scripts.convLSTMmodel import *
from ML_DL_Project.Scripts.objectAttentionModelConvLSTM import *
from ML_DL_Project.Scripts.spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from torch.utils.tensorboard import SummaryWriter
#from ML_DL_Project.Scripts.makeDatasetRGB import *
from ML_DL_Project.Scripts.makeMmaps import *
import argparse
import sys

In [8]:
# Dir for saving models and log files
model_folder = os.path.join('./', "out_dir", "./GTEA61", 'rgb')

if not os.path.isdir('./out_dir'):
  if os.path.exists(model_folder):
    print('Directory {} exists!'.format(model_folder))
    sys.exit()
    
  os.makedirs(model_folder)

writer = SummaryWriter(model_folder)
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')
train_log_loss_ss = open((model_folder + '/train_log_loss_ss.txt'), 'w')
train_log_acc_ss = open((model_folder + '/train_log_acc_ss.txt'), 'w')
val_log_loss_ss = open((model_folder + '/val_log_loss_ss.txt'), 'w')
val_log_acc_ss = open((model_folder + '/val_log_acc_ss.txt'), 'w')

In [10]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize, shuffle=True, num_workers=4, pin_memory=True)

if val_data_dir is not None:
  vid_seq_val = makeDataset(val_data_dir,spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                            seqLen=SEQLEN, fmt='.png')

  val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize, shuffle=False, num_workers=2, pin_memory=True)
  valInstances = vid_seq_val.__len__()


trainInstances = vid_seq_train.__len__()


In [11]:
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256),
                             RandomHorizontalFlip(), 
                             MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                             ToTensor(), 
                             normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=SEQLEN, fmt='.png')
vid_seq_val = makeDataset(val_data_dir, spatial_transform = Compose([Scale(256),
                                                                    CenterCrop(224),
                                                                    ToTensor(),
                                                                    normalize]),
                          seqLen=SEQLEN, fmt='.png')

In [18]:
def train(stage, learning_rate, numEpochs, decay_step, regressor, stage1_dict=None):
  train_params = []
  if stage == 1:
    model = attentionModel(num_classes=NUM_CLASSES, mem_size=MEMSIZE)
    model.train(False)
    for params in model.parameters():
        params.requires_grad = False
  else:
    model = attentionModel(num_classes=NUM_CLASSES, mem_size=MEMSIZE)
    model.load_state_dict(torch.load(stage1_dict))
    model.train(False)

    for params in model.parameters():
          params.requires_grad = False
      #
    for params in model.resNet.layer4[0].conv1.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[0].conv2.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[1].conv1.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[1].conv2.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.resNet.layer4[2].conv1.parameters():
        params.requires_grad = True
        train_params += [params]
      #
    for params in model.resNet.layer4[2].conv2.parameters():
        params.requires_grad = True
        train_params += [params]
      #
    for params in model.resNet.fc.parameters():
        params.requires_grad = True
        train_params += [params]

    model.resNet.layer4[0].conv1.train(True)
    model.resNet.layer4[0].conv2.train(True)
    model.resNet.layer4[1].conv1.train(True)
    model.resNet.layer4[1].conv2.train(True)
    model.resNet.layer4[2].conv1.train(True)
    model.resNet.layer4[2].conv2.train(True)
    model.resNet.fc.train(True)

  for params in model.lstm_cell.parameters():
      params.requires_grad = True
      train_params += [params]

  for params in model.classifier.parameters():
      params.requires_grad = True
      train_params += [params]
  model.lstm_cell.train(True)

  model.classifier.train(True)

  model = model.to(DEVICE)

  loss_fn = nn.CrossEntropyLoss()
  loss_regression= nn.MSELoss()
  loss_ms=nn.NLLLoss()

  optimizer_fn = torch.optim.Adam(train_params, lr=learning_rate, weight_decay=4e-5, eps=1e-4)

  optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step, gamma=decay_factor)
  train_iter = 0
  min_accuracy = 0

  for epoch in range(numEpochs):
    epoch_loss = 0
    epoch_loss_ss = 0
    numCorrTrain = 0
    numCorrTrain_ss = 0
    trainSamples = 0
    iterPerEpoch = 0
    model.lstm_cell.train(True)
    model.classifier.train(True)
    writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)

    if stage == 2:
        model.resNet.layer4[0].conv1.train(True)
        model.resNet.layer4[0].conv2.train(True)
        model.resNet.layer4[1].conv1.train(True)
        model.resNet.layer4[1].conv2.train(True)
        model.resNet.layer4[2].conv1.train(True)
        model.resNet.layer4[2].conv2.train(True)
        model.resNet.fc.train(True)
    for i, (inputs, maps, targets) in enumerate(train_loader):
      train_iter += 1
      iterPerEpoch += 1
      optimizer_fn.zero_grad()
      inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).to(DEVICE))
      labelVariable = Variable(targets.to(DEVICE))
      trainSamples += inputs.size(0)
      output_label, output_ss= model(inputVariable)
      loss = loss_fn(output_label, labelVariable)
      loss.backward()
      if regressor==0:
        maps=Variable(maps.permute(1, 0, 2, 3, 4).type(torch.LongTensor).to(DEVICE))
        output_ss= output_ss.view(-1, 2)
      if regressor==1:
        maps=Variable(maps.permute(1, 0, 2, 3, 4).to(DEVICE))
        output_ss=output_ss.view(-1)
      maps=maps.contiguos().view(-1)
      #print(maps)
      #Print for debugging

      if regressor==1:
        #Yes regression, the loss is obtained from the MSE error
        loss_ms=loss_regression(output_ss, maps)
        loss_ms.backward()
      if regressor==0:
        #No regression, the loss is computed by the cross entropy
        loss_ms=loss_fn(output_ss, maps)
        loss_ms.backward()
        _, predicted = torch.max(output_ms.data, 1)
        numCorrTrain_ss += torch.sum(predicted == maps.data).data.item()
        epoch_loss_ss+=loss_ms.item()
      optimizer_fn.step()
      _, predicted = torch.max(output_label.data, 1)
      numCorrTrain += torch.sum(predicted == labelVariable.data).data.item()
      epoch_loss+=loss.item()
      avg_loss = epoch_loss/iterPerEpoch

      if val_data_dir is not None:
        if (epoch+1) % 1 == 0:
            model.train(False)
            val_loss_epoch = 0
            val_loss_epoch_ss = 0
            val_iter = 0
            val_samples = 0
            numCorr = 0
            numCorr_ss = 0

            with torch.no_grad():
              for j, (inputs, maps, targets) in enumerate(val_loader):

                  val_iter += 1
                  val_samples += inputs.size(0)
                  inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
                  labelVariable = targets.to(DEVICE)
                  output_label, output_ss = model(inputVariable)
                  val_loss = loss_fn(output_label, labelVariable)
                  val_loss_epoch += val_loss.item()

                  if regressor==0:
                    maps=Variable(maps.permute(1, 0, 2, 3, 4).type(torch.LongTensor).to(DEVICE))
                    output_ss= output_ss.view(-1, 2)
                  if regressor==1:
                    maps=Variable(maps.permute(1, 0, 2, 3, 4).to(DEVICE))
                    output_ss=output_ss.view(-1)
                  maps=maps.contiguos().view(-1)
                  if regressor==1:
                    #Yes regression, the loss is obtained from the MSE error
                    loss_ms=loss_regression(output_ss, maps)
                    loss_ms.backward()
                  elif regressor==0:
                    #No regression, the loss is computed by the cross entropy
                    loss_ms=loss_fn(output_ss, maps)
                    loss_ms.backward()
                    _, predicted = torch.max(output_ms.data, 1)
                    numCorr_ss += torch.sum(predicted == maps.data).data.item()
                    epoch_loss_ss+=loss_ms.item()

                  _, predicted = torch.max(output_label.data, 1)
                  numCorr += torch.sum(predicted == labelVariable.data).data.item()

              avg_val_loss = val_loss_epoch / val_iter
              if stage ==2:
                    avg_loss_ms= epoch_loss_ss_val/ val_iter
                    val_accuracy = (numCorr_ss / val_samples) * 100
                    #avg_loss = avg_loss + avg_loss_ms 
                    #val_log_loss_ms.write('Val Loss MS after {} epochs = {}\n'.format(epoch + 1, avg_loss_ms))
                    if regressor == 0:
                      val_log_acc_ms.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))
              val_accuracy = (numCorr / val_samples) * 100
              print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))

              if val_accuracy > min_accuracy:
                save_path_model = (model_folder + '/model_rgb_state_dict.pth')
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
              elif (epoch+1) % 10 == 0:
                save_path_model = (model_folder + '/model_rgb_state_dict_epoch' + str(epoch+1) + '.pth')
                torch.save(model.state_dict(), save_path_model)


                  












In [ ]:
#Demo for contiguos
#aaa = torch.Tensor( [[1,2,3],[4,5,6]] )
#print(aaa.stride())
#print(aaa.is_contiguous())
#bbb = aaa.transpose(0,1)
#print(bbb)
#print(bbb.is_contiguous())
#bbb.contiguous().view(-1,3)

In [ ]:
#Demo for view reshaping
#x = torch.arange(6)
#print(x.view(-1,2))

tensor([[0, 1],
        [2, 3],
        [4, 5]])


In [ ]:
#Self-supervised without regressor
train(1, lr1, numEpochs1, 0, decay_step1)

train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
#writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.flush()
writer.close()


In [ ]:
#Self supervised with regressor
writer = SummaryWriter('runs/exp1/stage/2')
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')


train(2, lr2, numEpochs2, decay_step2, 1, stage1_dict="./out_dir/GTEA61/rgb/model_rgb_state_dict.pth")

train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
#writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.flush()
writer.close()

%tensorboard --logdir=runs/exp1
#!rm -rf ./runs/exp1/stage
#Using it only if run again, in view to clean the folders